In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix

In [4]:
d = pd.read_csv("../dats/battleship.csv")
d[d > 0] = 1

In [5]:
# create all combs
xx, yy = np.meshgrid(
    np.array(["A", "B", "C", "D", "E", "F", "G", "H"]),
    np.array([1, 2, 3, 4, 5, 6, 7, 8]),
    indexing="ij")
# create labels
labs = np.empty(xx.shape[0] * xx.shape[1], 
                dtype = "U2").reshape(xx.shape)
for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        labs[i, j] = xx[i,j] + str(yy[i, j])
labs = labs.flatten()

In [4]:
# # NOT RUN:
# y = d.loc[:, "D2"].values
# X = d.drop("D2", axis = 1).values
# par_grid = [
#     {"n_neighbors": [3, 5, 7, 10, 15]}
# ]
# m = KNeighborsClassifier()
# gs_cv = GridSearchCV(m, par_grid, 
#                      cv=5, 
#                      return_train_score=True,
#                      n_jobs=-4)
# gs_cv.fit(X, y)

In [6]:
mods = {}
for lab in labs:
    # select data
    y = d.loc[:, lab].values
    X = d.drop(lab, axis = 1).values
    # fit a model
    clf = RandomForestClassifier(max_features="sqrt", 
                                 max_depth=None, 
                                 min_samples_split=2,
                                 n_estimators=100,
                                n_jobs=-5)
    
    clf.fit(X, y)
    # store
    mods[lab] = clf
    

In [5]:
# # NOT RUN:
# mods = {}
# for lab in labs:
#     # select data
#     y = d.loc[:, lab].values
#     X = d.drop(lab, axis = 1).values
#     # fit a model
#     rfc = RandomForestClassifier(max_features="sqrt", 
#                                  max_depth=None, 
#                                  min_samples_split=2,
#                                  n_estimators=100,
#                                 n_jobs=3)
    
#     lgr = LogisticRegression(n_jobs=3)
#     gnb = GaussianNB()
#     clf = VotingClassifier(
#         estimators=[("rfc", rfc), ("lgr", lgr), ("gnb", gnb)],
#         voting="soft",
#         n_jobs=-4
#     )
#     clf.fit(X, y)
#     # store
#     mods[lab] = clf
    

In [ ]:
# import pickle
# with open("mods.pickle", "wb") as f:
#    pickle.dump(mods, f, protocol=5)

In [26]:
board=np.array(
    #1,2,3,4,5,6,7,8
    [0,0,0,0,0,0,0,0, #A
     0,0,0,0,0,0,0,1, #B
     0,1,1,1,1,1,0,1, #C
     0,1,0,0,0,0,0,1, #D
     0,1,0,1,0,0,0,1, #E
     0,0,0,1,0,1,0,0, #F
     0,0,0,1,0,1,0,0, #G
     0,0,0,0,0,0,0,0] #H
)

to_predict = labs[board == 0]
pred_prob = {}
for lab in to_predict:
    i = np.where(labs != lab)
    X_ = board[i]
    pred_prob[lab] = mods[lab].predict_proba(X_.reshape(1,-1))[0][1]

pd.Series(pred_prob).sort_values(ascending=False).head(30)

E6    0.62
H6    0.61
A8    0.59
F2    0.57
D4    0.48
F8    0.48
H4    0.46
E3    0.33
B2    0.33
G5    0.32
F7    0.25
C7    0.25
F5    0.23
D6    0.17
D3    0.15
C1    0.15
G7    0.12
E5    0.10
D1    0.10
E1    0.09
G2    0.08
G3    0.07
B6    0.05
F3    0.05
D5    0.04
G1    0.04
G8    0.03
D7    0.02
B5    0.02
E7    0.02
dtype: float64